In [1]:
import pandas as pd

# Load datasets
chn_2016 = pd.read_csv("data/CHN_STIR_Canada_2016.csv")
chn_2021 = pd.read_csv("data/CHN_STIR_Canada_2021.csv")
vacancy = pd.read_csv("data/CMHC_Vacancy.csv")
starts = pd.read_csv("data/Housing_Starts.csv")

# Quick checks
print("CHN 2016:")
display(chn_2016.head(), chn_2016.info())

print("\nCHN 2021:")
display(chn_2021.head(), chn_2021.info())

print("\nVacancy Rates:")
display(vacancy.head(), vacancy.info())

print("\nHousing Starts:")
display(starts.head(), starts.info())


FileNotFoundError: [Errno 2] No such file or directory: 'data/CHN_STIR_Canada_2016.csv'

In [2]:
import pandas as pd

# Load datasets (note the ../ to go up one folder)
chn_2016 = pd.read_csv("../data/CHN_STIR_Canada_2016.csv")
chn_2021 = pd.read_csv("../data/CHN_STIR_Canada_2021.csv")
vacancy = pd.read_csv("../data/CMHC Vacancy-eng.csv")
starts = pd.read_csv("../data/Housing starts-eng.csv")

# Quick overview
print("CHN 2016")
display(chn_2016.head(), chn_2016.info())

print("\nCHN 2021")
display(chn_2021.head(), chn_2021.info())

print("\nVacancy Rates")
display(vacancy.head(), vacancy.info())

print("\nHousing Starts")
display(starts.head(), starts.info())


ParserError: Error tokenizing data. C error: Expected 1 fields in line 9, saw 5


In [3]:
import pandas as pd
import io, re

def smart_read_csv(path, expected_header_keywords=("REF_DATE","Reference period","Year","GEO","Geography","Vector","VALUE","Value")):
    """
    Loads messy CSVs by:
      1) opening as raw text
      2) auto-detecting header row that contains known header keywords
      3) auto-detecting delimiter
      4) handling UTF-8 BOM and bad lines
    Returns: DataFrame
    """
    # Read raw text with common encodings
    for enc in ("utf-8-sig","utf-8","latin1"):
        try:
            with open(path, "r", encoding=enc, errors="ignore") as f:
                raw = f.read()
            break
        except Exception:
            continue

    # Split lines and locate header row
    lines = raw.splitlines()
    header_idx = None
    header_pattern = re.compile("|".join(re.escape(k) for k in expected_header_keywords), re.IGNORECASE)
    for i, line in enumerate(lines[:200]):  # scan first 200 lines for safety
        if header_pattern.search(line):
            header_idx = i
            break
    if header_idx is None:
        header_idx = 0  # fall back

    # Build a buffer starting at header row
    trimmed = "\n".join(lines[header_idx:])

    # Try auto delimiter first
    try:
        df = pd.read_csv(io.StringIO(trimmed),
                         engine="python",    # more tolerant
                         sep=None,           # auto-detect delimiter
                         on_bad_lines="skip")
        return df
    except Exception:
        pass

    # Try common delimiters explicitly
    for sep in [",",";","\t","|"]:
        try:
            df = pd.read_csv(io.StringIO(trimmed),
                             engine="python",
                             sep=sep,
                             on_bad_lines="skip")
            return df
        except Exception:
            continue

    # Final fallback: read with no header, then set first row as header
    df = pd.read_csv(io.StringIO(trimmed),
                     engine="python",
                     header=None,
                     on_bad_lines="skip")
    df.columns = df.iloc[0].astype(str)
    df = df.iloc[1:].reset_index(drop=True)
    return df

# --- Use the helper on your files (note the ../ paths from notebooks folder) ---
chn_2016 = smart_read_csv("../data/CHN_STIR_Canada_2016.csv")
chn_2021 = smart_read_csv("../data/CHN_STIR_Canada_2021.csv")
vacancy  = smart_read_csv("../data/CMHC Vacancy-eng.csv")      # or ../data/CMHC_Vacancy.csv if you renamed
starts   = smart_read_csv("../data/Housing starts-eng.csv")    # or ../data/Housing_Starts.csv

# Quick sanity checks
print("CHN 2016 columns:", list(chn_2016.columns)[:12])
print("CHN 2021 columns:", list(chn_2021.columns)[:12])
print("Vacancy columns:", list(vacancy.columns)[:12])
print("Starts columns:", list(starts.columns)[:12])

display(chn_2016.head(3))
display(chn_2021.head(3))
display(vacancy.head(3))
display(starts.head(3))


CHN 2016 columns: ['Geography: Canada, Province or territory, Census division, Census subdivision']
CHN 2021 columns: ['Geography: Canada, Province or territory, Census division, Census subdivision']
Vacancy columns: ['Geography: Census metropolitan area, Census metropolitan area part']
Starts columns: ['Geography: Canada, Geographical region of Canada, Province or territory']


,"Geography: Canada, Province or territory, Census division, Census subdivision"
0,Universe: Private households in occupied priva...
1,"Variable List: Housing indicators (9), Census ..."
2,Housing indicators (9) 3


,"Geography: Canada, Province or territory, Census division, Census subdivision"
0,Universe: Private households in occupied priva...
1,"Variable List: Housing indicators (9), Census ..."
2,Housing indicators (9) 3


,,,,,,,,,,,,"Geography: Census metropolitan area, Census metropolitan area part"
Geography,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
NaN,Rate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Census metropolitan areas,2.7,2.7,3.3,3.4,2.7,2.2,2.1,3.4,3.4,1.9,1.6,2.4


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"Geography: Canada, Geographical region of Canada, Province or territory"
Geography,January 2013,February 2013,March 2013,April 2013,May 2013,June 2013,July 2013,August 2013,September 2013,October 2013,November 2013,December 2013,January 2014,February 2014,March 2014,April 2014,May 2014,June 2014,July 2014,August 2014,September 2014,October 2014,November 2014,December 2014,January 2015,February 2015,March 2015,April 2015,May 2015,June 2015,July 2015,August 2015,September 2015,October 2015,November 2015,December 2015,January 2016,February 2016,March 2016,April 2016,May 2016,June 2016,July 2016,August 2016,September 2016,October 2016,November 2016,December 2016,January 2017,February 2017,March 2017,April 2017,May 2017,June 2017,July 2017,August 2017,September 2017,October 2017,November 2017,December 2017,January 2018,February 2018,March 2018,April 2018,May 2018,June 2018,July 2018,August 2018,September 2018,October 2018,November 2018,December 2018,January 2019,February 2019,March 2019,April 2019,May 2019,June 2019,July 2019,August 2019,September 2019,October 2019,November 2019,December 2019,January 2020,February 2020,March 2020,April 2020,May 2020,June 2020,July 2020,August 2020,September 2020,October 2020,November 2020,December 2020,January 2021,February 2021,March 2021,April 2021,May 2021,June 2021,July 2021,August 2021,September 2021,October 2021,November 2021,December 2021,January 2022,February 2022,March 2022,April 2022,May 2022,June 2022,July 2022,August 2022,September 2022,October 2022,November 2022,December 2022,January 2023,February 2023,March 2023,April 2023,May 2023,June 2023,July 2023,August 2023,September 2023,October 2023,November 2023,December 2023,January 2024,February 2024,March 2024,April 2024,May 2024,June 2024,July 2024,August 2024,September 2024,October 2024,November 2024,December 2024,January 2025
NaN,Units,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Canada,164.443,184.818,181.990,179.311,195.145,191.620,197.290,187.634,194.821,199.944,193.840,185.521,185.190,191.756,162.746,198.320,195.657,195.014,202.591,195.869,191.724,179.739,193.390,177.296,187.115,147.197,192.597,181.815,199.576,201.871,192.203,216.496,224.455,195.835,212.762,171.248,177.219,213.427,201.172,191.120,191.982,218.070,199.191,187.631,215.556,194.172,185.786,202.693,206.567,208.686,245.562,212.852,198.484,209.111,229.448,227.274,218.566,220.517,248.331,214.309,216.760,229.432,221.226,211.956,196.246,241.899,201.553,197.116,193.498,213.713,224.465,214.990,200.484,163.048,192.596,229.417,196.581,242.624,219.724,225.555,225.848,204.753,201.775,199.555,200.082,209.589,189.117,161.004,192.551,213.167,244.851,263.630,214.476,227.492,259.934,240.421,291.290,260.922,319.843,272.344,283.092,283.769,274.472,259.630,250.714,236.588,305.206,247.179,229.125,248.877,245.690,262.693,280.961,273.565,277.195,266.521,294.615,261.632,259.518,252.115,209.122,245.635,213.410,260.213,197.016,281.644,252.824,247.320,265.197,270.621,208.445,250.576,231.158,259.921,242.296,241.565,268.069,241.407,275.534,212.923,224.089,245.019,267.301,231.614,233.414
